---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [11]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind 

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [12]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [13]:
def get_list_of_university_towns():
    df = pd.DataFrame([], columns=['State', 'RegionName']) 
    with open('university_towns.txt', 'r') as f:
        state = ""
        for line in f:
            if '[edit]' in line:
                state = line[:line.find('[')].strip()
                continue
            region = line.strip()
            if '(' in region:
                region = region[:(region.find('(') - 1)]
            df = df.append(pd.DataFrame([[state, region]], columns=['State', 'RegionName']), ignore_index=True)
    return df
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [14]:
gdp=pd.read_excel('gdplev.xls')
gdp=gdp.drop(['Current-Dollar and "Real" Gross Domestic Product','Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 5'],axis=1)
gdp=gdp[7:]
gdp=gdp.rename(columns={'Unnamed: 4': 'Quarter', 'Unnamed: 6': 'GDP'})
gdp=gdp.set_index('Quarter')
index = gdp.index.get_loc('2000q1')
gdp = gdp.ix[index:]


In [15]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    for i in range(1, len(gdp) - 1):
        if (gdp.iloc[i]['GDP'] < gdp.iloc[i - 1]['GDP']) and (gdp.iloc[i + 1]['GDP'] < gdp.iloc[i]['GDP']):
            return gdp.iloc[i].name
    return None
    
get_recession_start()

'2008q3'

In [16]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    recession_start = get_recession_start()
    index = gdp.index.get_loc(recession_start)
    for i in range(index + 2, len(gdp)):
        if (gdp.iloc[i]['GDP'] > gdp.iloc[i - 1]['GDP']) and (gdp.iloc[i - 1]['GDP'] > gdp.iloc[i - 2]['GDP']):
            return gdp.iloc[i].name
    return None
get_recession_end()

'2009q4'

In [17]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    start = gdp.index.get_loc(get_recession_start())
    end = gdp.index.get_loc(get_recession_end())
    table = gdp['GDP'][start:end + 1]
    year = gdp[gdp['GDP'] == np.min(table)].iloc[0].name
    return year
get_recession_bottom()

'2009q2'

In [18]:
def get_info():
    df1=pd.read_csv('City_Zhvi_AllHomes.csv')
    df1=df1.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'],axis=1)
    df1=df1.replace({'State': states})
    df1=df1.set_index(["State","RegionName"])
    df1=df1.replace(to_replace='NaN', value=np.NaN)
    df1=df1.convert_objects(convert_numeric=True)
    index = list(df1.columns.values).index('2000-01')
    df1 = df1.drop(df1.columns[:index], axis=1)
    df1=df1.sort()
    return df1
get_info()

2000-01   2000-02   2000-03   2000-04   2000-05  \
State     RegionName                                                          
Alabama   Adamsville        68700.0   69100.0   69300.0   69200.0   69100.0   
          Alabaster        121600.0  122300.0  122500.0  122800.0  123200.0   
          Albertville       74300.0   74100.0   73500.0   72700.0   72500.0   
          Arab              84500.0   83900.0   82900.0   81900.0   81400.0   
          Ardmore               NaN       NaN       NaN       NaN       NaN   
          Axis                  NaN       NaN       NaN       NaN       NaN   
          Baileyton             NaN       NaN       NaN       NaN       NaN   
          Bay Minette       82900.0   81700.0   80500.0   79200.0   78200.0   
          Bayou La Batre    44000.0   44100.0   44100.0   44200.0   44600.0   
          Bessemer              NaN       NaN       NaN       NaN       NaN   
          Birmingham        53900.0   54000.0   54200.0   54300.0   54400.0   
          Boaz              70900.0   70900.0   70800.0   70400.0   70200.0   
          Brent             92500.0   92800.0   93500.0   94400.0   94400.0   
          Brighton              NaN       NaN       NaN       NaN       NaN   
          Brookwood         92500.0   92200.0   93000.0   94300.0   95000.0   
          Buhl              90900.0   90400.0   91100.0   93100.0   94900.0   
          Calera           108300.0  109300.0  109200.0  109900.0  110900.0   
          Center Point      80800.0   81000.0   81100.0   80900.0   81200.0   
          Centreville       95200.0   95000.0   95700.0   96800.0   96700.0   
          Chalkville        93900.0   94100.0   94300.0   94300.0   94500.0   
          Chancellor            NaN       NaN       NaN       NaN       NaN   
          Chelsea          160000.0  161800.0  164400.0  166600.0  167200.0   
          Chickasaw         50300.0   51200.0   52100.0   53100.0   53800.0   
          Chunchula         79800.0   80200.0   80800.0   81800.0   81800.0   
          Citronelle        63900.0   64700.0   65900.0   66700.0   66400.0   
          Clay             120400.0  121100.0  121200.0  121400.0  122200.0   
          Coden             62300.0   62500.0   63000.0   64100.0   65200.0   
          Coker            116400.0  118000.0  119900.0  121300.0  121000.0   
          Concord           78800.0   78400.0   78600.0   78500.0   78600.0   
          Cottondale       100500.0  100800.0  101200.0  102000.0  102700.0   
...                             ...       ...       ...       ...       ...   
Wisconsin Vernon           183200.0  183800.0  182600.0  180500.0  178100.0   
          Vienna           177100.0  177800.0  179200.0  181200.0  181800.0   
          Vinland          117900.0  119600.0  121900.0  124400.0  126600.0   
          Wales                 NaN       NaN       NaN       NaN       NaN   
          Waterford        122000.0  121000.0  120600.0  119900.0  119200.0   
          Waukesha         141400.0  141700.0  140700.0  139700.0  138400.0   
          Waunakee         186100.0  187400.0  188700.0  190200.0  191800.0   
          Waupun            83700.0   84200.0   84100.0   83900.0   83900.0   
          Wausau            68500.0   69800.0   70400.0   70000.0   69200.0   
          Wayne                 NaN       NaN       NaN       NaN       NaN   
          West Allis        80600.0   80900.0   81300.0   81600.0   81700.0   
          Weston            79200.0   79900.0   81400.0   82300.0   82700.0   
          Wheatland        128800.0  129500.0  131100.0  131800.0  131800.0   
          Whitelaw          94800.0   96100.0   97200.0   98800.0  101200.0   
          Williams Bay     127100.0  122600.0  119000.0  116900.0  115400.0   
          Wilson           128100.0  129200.0  129800.0  129300.0  129000.0   
          Wilson                NaN       NaN       NaN       NaN       NaN   
          Wind Lake        125200.0  124000.0  123900.0  123500.0  123400.0   

In [19]:
def get_quarter(year, month):
    if month <= 3:
        quarter = 1
    elif month <= 6:
        quarter = 2
    elif month <= 9:
        quarter = 3
    elif month <= 12:
        quarter = 4
    return (str(year) + 'q' + str(quarter))

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df1=get_info()
    l = len(df1.columns)
    i = 0
    while i < l:
        col_name = df1.iloc[:, i].name
        
        year = int(col_name.split('-')[0])
        month = int(col_name.split('-')[1])
        quarter = get_quarter(year, month)
        if i + 3 < l:
            split = df1.iloc[:, i:i + 3]
        else:
            split = df1.iloc[:, i:l]
        df1[quarter] = split.mean(axis=1)
        i += 3
    df1 = df1.drop(df1.columns[:l], axis=1)
    return df1
convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State     RegionName                                                     
Alabama   Adamsville        69033.333333   69166.666667   69800.000000   
          Alabaster        122133.333333  123066.666667  123166.666667   
          Albertville       73966.666667   72600.000000   72833.333333   
          Arab              83766.666667   81566.666667   81333.333333   
          Ardmore                    NaN            NaN            NaN   
          Axis                       NaN            NaN            NaN   
          Baileyton                  NaN            NaN            NaN   
          Bay Minette       81700.000000   78533.333333   79133.333333   
          Bayou La Batre    44066.666667   44500.000000   44266.666667   
          Bessemer                   NaN            NaN            NaN   
          Birmingham        54033.333333   54400.000000   54966.666667   
          Boaz              70866.666667   70266.666667   70300.000000   
          Brent             92933.333333   94333.333333   96166.666667   
          Brighton                   NaN            NaN            NaN   
          Brookwood         92566.666667   95100.000000   98866.666667   
          Buhl              90800.000000   94600.000000   96500.000000   
          Calera           108933.333333  110366.666667  108000.000000   
          Center Point      80966.666667   81233.333333   81500.000000   
          Centreville       95300.000000   96566.666667   98000.000000   
          Chalkville        94100.000000   94433.333333   94433.333333   
          Chancellor                 NaN            NaN            NaN   
          Chelsea          162066.666667  167033.333333  166900.000000   
          Chickasaw         51200.000000   53666.666667   54933.333333   
          Chunchula         80266.666667   81766.666667   82200.000000   
          Citronelle        64833.333333   66633.333333   68066.666667   
          Clay             120900.000000  122266.666667  123966.666667   
          Coden             62600.000000   64800.000000   66866.666667   
          Coker            118100.000000  120766.666667  118166.666667   
          Concord           78600.000000   78700.000000   80133.333333   
          Cottondale       100833.333333  102633.333333  104766.666667   
...                                  ...            ...            ...   
Wisconsin Vernon           183200.000000  178200.000000  174300.000000   
          Vienna           178033.333333  181533.333333  182433.333333   
          Vinland          119800.000000  126766.666667  134933.333333   
          Wales                      NaN            NaN            NaN   
          Waterford        121200.000000  119433.333333  120200.000000   
          Waukesha         141266.666667  138433.333333  136733.333333   
          Waunakee         187400.000000  191433.333333  192666.666667   
          Waupun            84000.000000   84000.000000   84400.000000   
          Wausau            69566.666667   69466.666667   71033.333333   
          Wayne                      NaN            NaN            NaN   
          West Allis        80933.333333   81933.333333   84066.666667   
          Weston            80166.666667   82700.000000   84166.666667   
          Wheatland        129800.000000  132133.333333  134933.333333   
          Whitelaw          96033.333333  101433.333333  108033.333333   
          Williams Bay     122900.000000  115300.000000  110766.666667   
          Wilson           129033.333333  129366.666667  132433.333333   
          Wilson                     NaN            NaN            NaN   
          Wind Lake        124366.666667  123666.666667  124133.333333   
          Wind Point       149233.333333  145266.666667  140866.666667   
          Wisconsin Dells  100000.000000  103400.000000  105000.000000   
          Wolf River        95166.666667   99333.333333  103800.000000   
          Woodruff                   N

In [20]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    hdf = convert_housing_data_to_quarters()
    start_index = hdf.columns.get_loc(get_recession_start())
    bottom_index = hdf.columns.get_loc(get_recession_bottom())
    hdf['Ratio'] = hdf.iloc[:, start_index - 1] / hdf.iloc[:, bottom_index]
    hdf = pd.DataFrame(hdf.loc[:, 'Ratio'])
    ul = get_list_of_university_towns()
    ul = ul.set_index(['State', 'RegionName'])
    univ_prices = pd.merge(hdf, ul, how="inner", left_index=True, right_index=True)
    non_univ = pd.merge(hdf, ul, how="outer", left_index=True, right_index=True, indicator=True)
    non_univ = non_univ[non_univ['_merge'] == 'left_only']
    non_univ = non_univ.drop('_merge', axis=1)
    univ_prices = univ_prices.dropna()
    non_univ = non_univ.dropna()
    s, p = ttest_ind(univ_prices['Ratio'], non_univ['Ratio'])
    s2, p2 = ttest_ind(non_univ['Ratio'], univ_prices['Ratio'])
    ans = True, p, "university town"
    return ans

run_ttest()
    


(True, 0.002724063704761164, 'university town')